In [1]:
#!/home/admin/anaconda3/bin/python3
#   author:martinmhan@yahoo.com date:  21/06/2020
#   Copyright (C) <2020>  <Martin Mohan>
#
#   This program is free software; you can redistribute it and/or modify
#   it under the terms of the GNU General Public License as published by
#   the Free Software Foundation; either version 3 of the License, or
#   (at your option) any later version.
#
#   This program is distributed in the hope that it will be useful,
#   but WITHOUT ANY WARRANTY; without even the implied warranty of
#   MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#   GNU General Public License for more details.
#
#   You should have received a copy of the GNU General Public License
#   along with this program; if not, write to the Free Software Foundation,
#   Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301  USA
#print(__doc__)
#import joblib,argparse,re,sys,glob,os,time
import argparse,re,sys,glob,os,time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import sklearn.metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from itertools import cycle
import json
import mmutils,myfit
import rpt2tex
import mmodels

class Treat4():
    """ Given input file and model name select the correct classifier.
    Compare results roc,cm,metrics with train/test split of data.
    Run trained model against TCE file to predict new planets.

    """
    def __init__(self,ifile,fit,model):
        """ Given input file and model name select the correct classifier.
        Compare results roc,cm,metrics with train/test split of data.
        Run trained model against TCE file to predict new planets.

        """
        bTK=False
        if("data/bT" in ifile):
            bTK=True
            print(f'{ifile} contains data/b bTK = {bTK}')
            if (not model.startswith("b")):
                print(f'Warn binary class model name recommended "b" not {model}')
            else:
                print(f'Binary class {ifile} and {model}')
        else:
            print(f'Three classes for  {ifile} {model}')
        self.bTK=bTK
        self.ifile=ifile
        self.model=model
        self.myfit=myfit.myfit(self.ifile,fit,model)
        self.ofile=self.myfit.ofile # base for all other filenames
        print(f"ifile {self.ifile}, model {model}")
        self.clf=self.myfit.clf
#        self.save_clf() # Saves classifier in file as verbatim tex

    def save_clf(self):    
        a=mmodels.mmodels(argv.model)
        output="\\begin{verbatim}\n\
        %s\n\
        \\end{verbatim}" %(a.clf)
        ftex=self.ofile+"_clf.tex"
        with open(ftex,'w') as f: f.write(output)
        print(f"Saved Clf in {ftex}")

    def get_plnt_num(self,kepname):
        """ Called by getKepid to extract plnt_num from kepoi_name
            e.g. Extract K00082.01 -> ['1']

        """
        x=re.findall(r'\d$', kepname) # K00001.01 -> ['1']
        y=list(map(int, x)) # ['1'] -> 1 doesn't work?
        return int(y[0])
    
    # Merge data/KOI.csv and data/TCE.csv to data/TK.csv
    def getKepid(self,pfile,koi1):
        """ Generate Kepid. The KOI table contains kepid and kepoi_name (e.g. K00082.01)
        The TCE contains kepid and tce_plnt_num. This joins tables on kepid, tce_plnt_numb

        """
        for index, row in koi1.iterrows():
            koi1.loc[index,'tce_plnt_num'] = self.get_plnt_num(row['kepoi_name'])
        koi1.tce_plnt_num = koi1.tce_plnt_num.astype(int) # ['1.0'] -> ['1']
        df=koi1.merge(pfile, indicator=True, how='outer')
        df=df.drop(['_merge'], axis=1)
        return df

    def crefFile(self): # combine results for cross reference
        """ Join the input and the prediction file using Kepid and tce_plnt_num 
        Then generate precdictions and store with cross referenced results

        """

        ifilepred=self.ifile.replace("TK", "TCE1") # Prediction file must already exist and have same extensions as TK... TCE1_...
        df=pd.read_csv(self.ifile,comment="#")  # Reference file with DV
        df = df[['kepid','tce_plnt_num','kepoi_name','koi_disposition']].copy()
        ######################################################
        dfp=pd.read_csv(ifilepred,comment='#')
        X_pred=dfp.drop(['kepid'], axis=1).to_numpy()
        # Predict
        dfp1=pd.DataFrame(self.clf.predict_proba(X_pred), columns=self.clf.classes_) # clf.classes maintained order FP,CONF,CAND
        dfp1['y_pred']=self.clf.predict(X_pred)
        dfp1['kepid'] = dfp.kepid.astype(int)
        dfp1['tce_plnt_num']=dfp.tce_plnt_num.astype(int)
        #########################################################
        dfm=self.getKepid(dfp1,df)
        
        cref=self.ofile+"_cref.csv"
        dfm.to_csv(cref,index=False)
        print(f"Saved {cref}")
        return cref

    def plot_roc(self,df,force):
        """ Plot the ROC. If ifile and model begin with b it is binary not multiclass. """
        if self.bTK:
            style = {"CONFIRMED": "green","FALSE POSITIVE": "red"}
        else:
            style = {"CONFIRMED": "green","CANDIDATE": "darkorange","FALSE POSITIVE": "red"}
    
        plt.rcParams.update({'axes.labelsize': 'x-large'})
        fpr = dict();tpr = dict();roc_auc=dict();lw=2
    
        for i in style:
            #    fpr[i], tpr[i],_ =roc_curve(y_test, df[i], pos_label=i)
            fpr[i], tpr[i],_ =roc_curve(df['y_test'], df[i], pos_label=i)
            roc_auc[i] = auc(fpr[i], tpr[i])
            plt.plot(fpr[i], tpr[i],color=style[i], lw=lw,\
                    label='AUC: {0} vs REST (area = {1:0.2f})'.format(i, roc_auc[i]))
    
            #y_score = clf.decision_function(X_test)
        # Plot ROC curve
        plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate or (1 - Specificity)')
        plt.ylabel('True Positive Rate or (Sensitivity)')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        rocfile=self.ofile+'_roc.pdf'
        mmutils.write2plt(plt,rocfile,force)
        return rocfile

    def plot_cm(self,force):
        """ Plot the Confusion Matrix """
        # Confusion Matrix
        cm = pd.crosstab(df.y_test, df.y_pred, rownames=['Actual'], colnames=['Predicted'])
        print(cm)
            #cm = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])
        fig, (ax) = plt.subplots(1)
        sns.heatmap(cm,annot=True,cmap='Blues', fmt='g')
    
        # labels, title and ticks
        ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
        ax.set_title('Confusion Matrix');
        if self.bTK:
            ax.xaxis.set_ticklabels(['CONFIRMED', 'FALSE POSITIVE']);
        else:
            ax.xaxis.set_ticklabels(['CANDIDATE','CONFIRMED', 'FALSE POSITIVE']);
        cmfile=self.ofile+'_cm.pdf'
        mmutils.write2plt(plt,cmfile,force)
        return cmfile
    
    def metric_csv(self,df,caption,force):
        " Generate latex reprot with roc,cm and metrics"""
        print(sklearn.metrics.classification_report(df.y_test, df.y_pred))
        report=sklearn.metrics.classification_report(df.y_test, df.y_pred,output_dict=True)
        report = pd.DataFrame(report).transpose()
        rptfile=self.ofile+"_rpt.csv"
        mmutils.write2csv(report,rptfile,force)
        return rptfile


    def div(self,x, y):
        """ If divsion by zero return 0 """
        return 0 if y == 0 else (x / y)*100

    def plot_cref(self,creffile):
        """Load and examine cross reference file with predictions. Generate bar plots of CONFIRMED vs Predicted """
        # Get Results
        myclass='CONFIRMED'
        df=pd.read_csv(creffile,comment='#')
        totCONFIRMED=df.loc[(df['koi_disposition'] == 'CONFIRMED')]
        totFP=df.loc[(df['koi_disposition'] == 'FALSE POSITIVE')]
        totCAND=df.loc[(df['koi_disposition'] == 'CANDIDATE')]
        totUN=df.loc[(df['koi_disposition'].isnull() ) ]
    
# recovered
        rCONFIRMED=df.loc[(df['y_pred'] ==  'CONFIRMED') & (df['koi_disposition'] ==  'CONFIRMED')]                                        
        rFP=df.loc[(df['y_pred'] == 'FP') & (df['koi_disposition'] ==  'CONFIRMED')]
        rCAND=df.loc[(df['y_pred'] == 'CANDIDATE') & (df['koi_disposition'] ==  'CONFIRMED')]
        rUN=df.loc[(df['y_pred'] == 'CONFIRMED') & (df['koi_disposition'].isnull() )]


        df = pd.DataFrame([[rCONFIRMED.shape[0],totCONFIRMED.shape[0], self.div(rCONFIRMED.shape[0],totCONFIRMED.shape[0])],\
                       [rFP.shape[0],totFP.shape[0], self.div(rFP.shape[0],totFP.shape[0])],\
                       [rCAND.shape[0],totCAND.shape[0], self.div(rCAND.shape[0],totCAND.shape[0])],\
                       [rUN.shape[0],totUN.shape[0], self.div(rUN.shape[0],totUN.shape[0])]],\
         index=['CONFIRMED','FALSE POSITIVE','CANDIDATE','TCE1'],
         columns=['recovered','total','Percentage'])

        # PLOT
        fig, axs = plt.subplots(2,2,sharex='col', sharey='row')
        title="Planets predicted as CONFIRMED"
    #    title=mmutils.models()[model].split(".")[-1] # 'sklearn.ensemble.GradientBoostingClassifier'
    
        bins=50
        dfy=rCONFIRMED
        axs[0,0].hist(dfy[myclass], bins=bins,color='green')
        pcnt=" ("+str(round(df.loc['CONFIRMED','Percentage'], 2))+"%)"
        axs[0,0].set_title( 'CONFIRMED '+ str(dfy.shape[0])+" / "+str(totCONFIRMED.shape[0]) + pcnt)
        axs[0,0].set_ylabel('Nr of Planets')

        dfy=rUN
        axs[0,1].hist(dfy[myclass], bins=bins,color='grey')
        pcnt=" ("+str(round(df.loc['TCE1','Percentage'], 2))+"%)"
        axs[0,1].set_title("TCE1 "+ str(dfy.shape[0])  + " / "+ str(totUN.shape[0])  + pcnt)

        dfy=rCAND
        axs[1,0].hist(dfy[myclass], bins=bins,color='darkorange')
        pcnt=" ("+str(round(df.loc['CANDIDATE','Percentage'], 2))+"%)"
        axs[1,0].set_title( 'CANDIDATE '+  str(dfy.shape[0])+" / "+  str(totCAND.shape[0])  + pcnt)
        axs[1,0].set_ylabel('Nr of Planets')
        axs[1,0].set(xlabel='Probability Score')
    
        dfy=rFP
        axs[1,1].hist(dfy[myclass], bins=bins,color='red')
        pcnt=" ("+str(round(df.loc['FALSE POSITIVE','Percentage'], 2))+"%)"
        axs[1,1].set_title("FALSE POSITIVE "+ str(dfy.shape[0]) +" / "+ str(totFP.shape[0])  + pcnt)
        axs[1,1].set(xlabel='Probability Score')

        for ax in axs.flat:
            ax.set_ylim(0,100)
            ax.set_xlim(0.3,1)

        # Create files for latex
        rptfile=creffile.replace(".csv","_rpt.csv")
#        df.to_csv(rptfile,index=False)
#        print(f"saved {rptfile}")

        myplot=rptfile.replace("rpt.csv",'CONFIRMED.pdf')
        plt.savefig(myplot)
        print(f"Saved {myplot}")
        return myplot

#        myrpt=rpt2tex.rpt2tex(rptfile) 
#        output=myrpt.creftex(argv.caption)
#        return output

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Run model and generate report files  _roc.pdf, _cm.pdf, _report.csv, _tex ',formatter_class=argparse.ArgumentDefaultsHelpFormatter)

    parser.add_argument( "--ifile", type=str, default="data/TK.csv",
            help="input file: train_test_split will be run on this file (default: %(default)s)")

    parser.add_argument( "--model", type=str, default="GB",
            help="Model to test (default: %(default)s)")

    parser.add_argument("--overfit", action="store_true",
            help="overfit test: Use train data to predict outcome and compare against test - ofile will have name _overfit_ ")

    parser.add_argument("--fit", action="store_true",
            help="fit model - otherwise load model from .pickle")

    parser.add_argument("--force", action="store_true",
            help="force overwrite of results")

    parser.add_argument( "--caption", type=str, default="",
            help="Caption to add to eventual tex file (must be latex friendly) (default: %(default)s)")

    parser.add_argument("--showmodels", action="store_true",
            help="Show all models and exit")

#    argv=parser.parse_args()
        # argv=parser.parse_args()
    
    class Args:
        ifile = 'data/TK.csv'
        model = "GB"
        fit = False
        force = False
        caption = ""
        showmodels=False
    argv=Args()

    if argv.showmodels:
        mymodel=mmodels.mmodels("GB").desc
        print(json.dumps(mymodel, indent=4, ensure_ascii=False))
        sys.exit(0)

### Start
    mytreat=Treat4(argv.ifile,argv.fit,argv.model)

    # Overfit test i.e. run model on train data / else test data
    if argv.overfit:
        mytreat.ofile=mytreat.ofile+"_overfit"
        df=mytreat.myfit.overfit_results()
    else:
        df=mytreat.myfit.predict_results()

# Generate graph from results show prediction accuracy
    froc=mytreat.plot_roc(df,argv.force)
    fcm=mytreat.plot_cm(argv.force)
    fcsv=mytreat.metric_csv(df,argv.caption,argv.force)
    caption=argv.caption

# Run model on prediction file to check recovery and find new planets
    cref=mytreat.crefFile() # Run model on prediction file
    fcref=mytreat.plot_cref(cref)
    myrpt=rpt2tex.rpt2tex(froc)

    modelname=argv.model.replace("_","")
    if argv.overfit:
        caption=f"{modelname}: overfit check "
        caption=f"{modelname}: Overfit check (train data). ROC Curve, Confusion Matrix"
        output=myrpt.p2(froc,fcm,caption)
    else:
        caption=f"{modelname}: ROC Curve, Confusion Matrix, Predicted CONFIRMED Recovery and Metrics"
        output=myrpt.p3c1(froc,fcm,fcref,fcsv,caption)


Three classes for  data/TK.csv GB


TypeError: __init__() takes from 2 to 3 positional arguments but 4 were given